https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=72b76cc09a800810VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [12]:
import pandas as pd

# Cargar el archivo CSV
lamps = pd.read_excel('20220408_DATOS_ABIERTOS_UNIDAD_LUMINOSA_.xlsx')

# Mostrar las primeras filas del DataFrame para verificar la carga correcta
display(lamps)


,"tipo_bloqu,C,50","TIPOL,C,254","VIA_CLASE,C,254","VIA_PAR,C,254","VIA_NOMBRE,C,254","CLASE_APP,C,254","NUMERO,N,10,0","COD_NDP,N,10,0","DISTRITO,N,10,0","BARRIO,N,10,0","X_UTM,N,19,11","Y_UTM,N,19,11"
0,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,3,20040016,16,4,445648.4812,4.480671e+06
1,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,10,31024246,16,4,445610.6416,4.480741e+06
2,GLED001,LED,PLAZA,DE,MAR DEL PLATA,NUMERO,12,11119324,16,4,445680.0750,4.480679e+06
3,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,21,11119327,16,4,445669.5353,4.480645e+06
4,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,19,11119326,16,4,445651.1627,4.480635e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
233304,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,4,31022348,16,4,445636.7220,4.480688e+06
233305,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,6,31022349,16,4,445626.8779,4.480711e+06
233306,LBE026,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,9,11119314,16,4,445612.3597,4.480727e+06
233307,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,9,11119314,16,4,445616.6395,4.480730e+06


In [14]:
from pyproj import Transformer

# Definir el transformador de UTM a latitud/longitud (WGS84)
# Asumo que las coordenadas UTM están en la zona 30 Norte (EPSG:32630). Ajusta la zona según tu caso.
transformer = Transformer.from_crs("EPSG:32630", "EPSG:4326", always_xy=True)

# Función para convertir UTM a latitud/longitud
def utm_to_latlon(easting, northing):
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

# Aplicar la transformación a las columnas UTM y crear nuevas columnas "latitud" y "longitud"
lamps[["latitud", "longitud"]] = lamps.apply(
    lambda row: utm_to_latlon(row["X_UTM,N,19,11"], row["Y_UTM,N,19,11"]), axis=1, result_type="expand"
)

  tipo_bloqu,C,50 TIPOL,C,254 VIA_CLASE,C,254 VIA_PAR,C,254 VIA_NOMBRE,C,254  \
0          LBE002    DESCARGA           CALLE        DE LAS             ERAS   
1          LBE002    DESCARGA           CALLE        DE LAS             ERAS   
2         GLED001         LED           PLAZA            DE    MAR DEL PLATA   
3        FFLED005         LED           CALLE           DEL     MAR AMARILLO   
4        FFLED005         LED           CALLE           DEL     MAR AMARILLO   

  CLASE_APP,C,254  NUMERO,N,10,0  COD_NDP,N,10,0  DISTRITO,N,10,0  \
0          NUMERO              3        20040016               16   
1          NUMERO             10        31024246               16   
2          NUMERO             12        11119324               16   
3          NUMERO             21        11119327               16   
4          NUMERO             19        11119326               16   

   BARRIO,N,10,0  X_UTM,N,19,11  Y_UTM,N,19,11    latitud  longitud  
0              4    445648.4812   

In [24]:
import pandas as pd
import geopandas as gpd
import folium
from pyproj import Transformer
from shapely.geometry import Point

# 1. Cargar y transformar coordenadas
farolas = pd.read_excel("20220408_DATOS_ABIERTOS_UNIDAD_LUMINOSA_.xlsx")
transformer = Transformer.from_crs("EPSG:32630", "EPSG:4326")
farolas['longitud'], farolas['latitud'] = transformer.transform(
    farolas["X_UTM,N,19,11"].values, 
    farolas["Y_UTM,N,19,11"].values
)

# 2. Crear GeoDataFrame de farolas
farolas_gdf = gpd.GeoDataFrame(
    farolas,
    geometry=gpd.points_from_xy(farolas.longitud, farolas.latitud),
    crs="EPSG:4326"
)

# 3. Cargar y limpiar distritos
distritos = gpd.read_file("madrid-districts.geojson").to_crs("EPSG:4326")
distritos_filtrados = distritos[distritos.name.isin(["Centro", "Salamanca", "Retiro"])]
distritos_filtrados = distritos_filtrados.drop(columns=['created_at', 'updated_at', 'cartodb_id'])  # Eliminar columnas problemáticas

# 4. Filtrar farolas en los distritos
farolas_filtradas = gpd.sjoin(farolas_gdf, distritos_filtrados, predicate="within")

# 5. Contar y mostrar resultados
conteo = farolas_filtradas.groupby('name').size()
print("Farolas por distrito:")
print(conteo.to_string())

# 6. Crear mapa básico
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=13)

# 7. Añadir puntos amarillos
for _, row in farolas_filtradas.iterrows():
    folium.CircleMarker(
        location=[row.latitud, row.longitud],
        radius=3,
        color='yellow',
        fill=True,
        fill_color='yellow'
    ).add_to(mapa)

# 8. Guardar y abrir
mapa.save("farolas_madrid.html")
print("\nMapa guardado correctamente como 'farolas_madrid.html'")

Farolas por distrito:
Series([], )

Mapa guardado correctamente como 'farolas_madrid.html'
